In [3]:
import ee
import pandas as pd

ee.Initialize(project='pranavkoka123')

geometry = ee.Geometry.Rectangle([78.97, 30.70, 79.20, 30.79])

# Calculate NDSI function
def calculate_ndsi(image):
    # NDSI = (Green - SWIR) / (Green + SWIR)
    green = image.select('B3')  # Green band
    swir = image.select('B6')  # SWIR band
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    return image.addBands(ndsi)

# Function to calculate the percentage of pixels with NDSI > 0.8 for a given month
def calculate_percentage_ndsi(month_start, month_end):
    # Filter the image collection for a given month
    image = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
                .filterDate(month_start, month_end) \
                .filterBounds(geometry) \
                .first()

    # Apply NDSI calculation
    image_ndsi = calculate_ndsi(image)

    # Create a mask for NDSI values greater than 0.4
    ndsi_greater_04 = image_ndsi.select('NDSI').gt(0.4)

    # Apply unmask to handle missing values as 0
    ndsi_greater_04 = ndsi_greater_04.unmask(0)

    # Count the total number of valid pixels (not masked out)
    total_pixels = image_ndsi.select('NDSI').unmask(0).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=30
    ).get('NDSI')

    # Count the number of pixels with NDSI > 0.4
    pixels_greater_04 = ndsi_greater_04.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=30
    ).get('NDSI')

    # Calculate the percentage of pixels with NDSI > 0.4
    percentage_greater_04 = ee.Number(pixels_greater_04).divide(total_pixels).multiply(100)

    return percentage_greater_04

# Define the months of the year 2015
months = [
    ('2015-01-01', '2015-01-31'),
    ('2015-02-01', '2015-02-28'),
    ('2015-03-01', '2015-03-31'),
    ('2015-04-01', '2015-04-30'),
    ('2015-05-01', '2015-05-31'),
    ('2015-06-01', '2015-06-30'),
    ('2015-07-01', '2015-07-31'),
    ('2015-08-01', '2015-08-31'),
    ('2015-09-01', '2015-09-30'),
    ('2015-10-01', '2015-10-31'),
    ('2015-11-01', '2015-11-30'),
    ('2015-12-01', '2015-12-31')
]

# Create an empty list to store the results
results_list = []

# Loop through each month and calculate the percentage of pixels with NDSI > 0.4
for i, (start, end) in enumerate(months):
    percentage = calculate_percentage_ndsi(start, end)
    percentage_value = percentage.getInfo()  # Get the value from Earth Engine
    results_list.append({'Month': f'{i+1}', 'Percentage': percentage_value})

# Create a pandas DataFrame from the results
df_percentages = pd.DataFrame(results_list)

# Display the DataFrame
print(df_percentages)


   Month  Percentage
0      1   98.945445
1      2   97.627247
2      3   91.770110
3      4   97.290345
4      5    1.870788
5      6    4.710272
6      7   46.309281
7      8    0.646767
8      9   26.515653
9     10   29.605611
10    11    1.408746
11    12   31.844057
